Imports

In [19]:
#imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, pipeline

from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from naive_extractor import naive_extraction
from bert_extractor import bert_extraction
from llama_extractor import llama_extraction, process_llama_response

Data Loading

In [20]:
# Load data
df = load_data("synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [21]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


Naive Extractor

In [22]:
#Process samples
predictions = []

for sample in samples:
    relationships = naive_extraction(sample['note_text'], sample['entities_list'], sample['dates'])
    predictions.extend(relationships)

In [23]:
# Look at predictions
#predictions

In [24]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.1277533039647577,
 'recall': 0.13551401869158877,
 'f1': 0.13151927437641722,
 'tp': 29,
 'fp': 198,
 'fn': 185}

BERT

In [25]:
# Load model and tokenizer
model_name = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Your extraction logic in notebook
predictions = []
for sample in samples:
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    
    for pair in pairs:
        text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text'][:200]}..."
        pred, conf = bert_extraction(text, model, tokenizer)
        
        if pred == 1:
            predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

In [27]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.1724137931034483,
 'recall': 0.02336448598130841,
 'f1': 0.0411522633744856,
 'tp': 5,
 'fp': 24,
 'fn': 209}

Llama

In [28]:
# Load from local directory
model_path = "../Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# Load in notebook (you control this)
generator = pipeline("text-generation", model="../Llama-3.2-3B-Instruct", device=-1)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [30]:
# Create prompt in notebook
#prompt = f"In the following text, confirm if {text} Answer YES or NO:"
#response = llama_extraction(prompt, generator)

In [31]:
#prompt

In [32]:
predictions = []

for sample in samples[:1]:
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])

    for pair in pairs[:1]:
        prompt = f"Does '{pair['entity_label']}' relate to '{pair['date']}' in this medical text? Answer YES or NO: {sample['note_text'][:200]}."
        
        # Get Llama response
        response = llama_extraction(prompt, generator)
        
        # Process response to get prediction
        prediction, confidence = process_llama_response(response)
        
        if prediction == 1:  # Only save positive predictions
            predictions.append({
                'entity_label': pair['entity_label'],
                'date': pair['date'],
                'confidence': confidence
            })

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [33]:
prompt

"Does 'asthma' relate to '2024-08-02' in this medical text? Answer YES or NO: Ultrasound (30nd Jun 2024): no significant findings.imp: asthma\n\nShe denies any nausea, vomiting, or diarrhea.\nC Patient reports compliance with current medication regimen. Basic metabolic panel withi."

In [34]:
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0, 'recall': 0.0, 'f1': 0, 'tp': 0, 'fp': 0, 'fn': 214}